<a href="https://colab.research.google.com/github/Emma922/Olist-Project/blob/main/Olist_EDA_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Exploratory Data Analysis & Data Cleaning – Olist E-Commerce Dataset

## 1. Introduction  
This notebook performs a complete **Exploratory Data Analysis (EDA)** and **Data Cleaning** of the Olist e-commerce dataset.  
The goal is to understand customer behavior, order patterns, payment habits, review performance, and product characteristics.

We will:

- Load all Olist datasets  
- Explore structure, quality, missing data  
- Clean and export the tables  
- Understand correlations
- Detect outliers  

---

# 2. Import Libraries


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("default")
sns.set_theme()


# 3. Load the Datasets

Below is a description of each file:

| File           | Description                      |
|:-------------- |:-------------------------------- |
| customers   | Customers information + location         |
| geolocation | Info of zip codes + lat/lng |
| orders_items        | Data about items purchased  |
| order_payments      | Order payments details           |
| order_reviews       | Data about reviews made by customers     |
| order      | Core dataset, Orders details   |
| products        | Products details     |
| sellers        | Sellers details      |






In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
customers = pd.read_csv("olist_customers_dataset.csv")
geolocation = pd.read_csv("olist_geolocation_dataset.csv")
orders_items = pd.read_csv("olist_order_items_dataset.csv")
order_payments = pd.read_csv("olist_order_payments_dataset.csv")
order_reviews = pd.read_csv("olist_order_reviews_dataset.csv")
order = pd.read_csv("olist_orders_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
sellers = pd.read_csv("olist_sellers_dataset.csv")

# 4. Preview & Raw Exploration

We begin by inspecting the shape, structure, head, and summary of each dataset.


In [ ]:
dfs = {
    "customers": customers,
    "geolocation": geolocation,
    "order_items": orders_items,
    "order_payments": order_payments,
    "order_reviews": order_reviews,
    "order": order,
    "products": products,
    "sellers": sellers
}

for name, df in dfs.items():
    print(f"======== {name.upper()} ========")

    display(df.head(3))

    print("\nInfo:")
    display(df.info())

    print("\nDescribe:")
    display(df.describe(include='all')
)

    print("\n\n")



# 5. Data Cleaning & Pre-processing

## 5.1 Missing values overview

In [ ]:
dfs = {
    "customers": customers,
    "geolocation": geolocation,
    "order_items": orders_items,
    "order_payments": order_payments,
    "order_reviews": order_reviews,
    "order": order,
    "products": products,
    "sellers": sellers
}

for name, df in dfs.items():
    print(f"\nNull count for {name}:")
    print(df.isnull().sum())


## 5.2 Missing Numeric Values
- We'll look whether the dataset have missing values

In [ ]:
dfs = {
    "customers": customers,
    "geolocation": geolocation,
    "order_items": orders_items,
    "order_payments": order_payments,
    "order_reviews": order_reviews,
    "order": order,
    "products": products,
    "sellers": sellers
}

for name, df in dfs.items():
    print(f"===== {name.upper()} =====")

    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    missing_numeric = df[numeric_cols].isna().sum()

    print("Numeric columns with missing values:")
    print(missing_numeric[missing_numeric > 0])

    print("\n")


## 5.3 Outliers
- An outlier analysis will be performed to identify unusually extreme values that could distort the results of the study.


In [ ]:
dfs = {
    "customers": customers,
    "geolocation": geolocation,
    "order_items": orders_items,
    "order_payments": order_payments,
    "order_reviews": order_reviews,
    "order": order,
    "products": products,
    "sellers": sellers
}

for name, df in dfs.items():
    numeric_df = df.select_dtypes(include=[float, int])
    corr = numeric_df.corr()

    if corr.empty:
        print(f"\nSkipping {name} because correlation matrix is empty or no numeric data.")
        continue

    # Plot boxplot for numeric columns to detect outliers
    plt.figure(figsize=(12, 6))
    numeric_df.boxplot()
    plt.title(f'Boxplot for Numeric Columns of {name}')
    plt.xticks(rotation=75)
    plt.show()


## 5.4 Duplicates Overview

In [ ]:
dfs = {
    "customers": customers,
    "geolocation": geolocation,
    "order_items": orders_items,
    "order_payments": order_payments,
    "order_reviews": order_reviews,
    "order": order,
    "products": products,
    "sellers": sellers
}

for name, df in dfs.items():
    print(f"\nDuplicate rows for {name}:")
    duplicates = df[df.duplicated()]
    if duplicates.empty:
        print("No duplicate rows found.")
    else:
        print(duplicates)


## 5.4.1 Handling Duplicates

- No duplicate records were identified in the majority of the tables, indicating consistent and well-structured data across most datasets.

- The geolocation table contains duplicate entries; however, these will be evaluated later during the SQL analysis to determine whether they represent legitimate repeated locations or redundant data.

- Therefore, no duplicate removal has been applied at this stage, ensuring that no potentially relevant information is discarded prematurely.

## 5.5 Handling Missing values

### Strategy:
- Datetime columns: Missing timestamps are preserved because they often carry operational meaning (e.g., cancellations, unprocessed orders). Imputing or replace them may cause biased and unreal results

- Categorical columns: Missing values are replaced with "Unknown" to maintain categorical integrity and avoid incorrect assumptions.

- Numeric columns: Rare missing values are imputed with the median; if missingness is widespread and the column is not essential, the feature may be removed.

- High-missingness irrelevant columns: Columns containing a high proportion of missing data and not neccesary to the analysis will be dropped



## 5.5.1 Drop unnecesary columns
- Columns that won't be neccesary for the analysis and contain a high propotion of missing data


In [ ]:
order_reviews = order_reviews.drop(
    columns=[
        'review_comment_title',
        'review_comment_message',
        'review_creation_date',
        'review_answer_timestamp'
    ]
)
products = products.drop(
    columns=[
        'product_name_lenght',
        'product_description_lenght',
        'product_photos_qty'
    ]
)

## 5.5.2 Replace missing values
- Missing values are replaced by 0 in products measures


In [ ]:
products['product_weight_g'] = products['product_weight_g'].fillna(0)
products['product_length_cm'] = products['product_length_cm'].fillna(0)
products['product_height_cm'] = products['product_height_cm'].fillna(0)
products['product_width_cm'] = products['product_width_cm'].fillna(0)

## 5.5.3 Replace or imput numeric missing values
After reviewing all datasets, no numeric columns contain missing values across any of the tables. This indicates that:

- All numeric attributes are fully populated

- No median imputation is required

- No numeric features need to be removed due to sparsity

## 5.5.4 Replace outliers with avg values

- The boxplot analysis shows no presence of significant outliers across the numeric variables.

- All detected values fall within expected and reasonable ranges for the dataset.

- Although some variables contain large numerical values, a closer inspection shows that these values are consistent with the nature of the dataset and represent realistic business scenarios

- Because of this, no outlier treatment is required

## 5.5.5 Cleaning missings datetime values

- Datetime features often carry important chronological patterns that are crucial for understanding customer behavior, delivery timelines, and order processing.

- Imputing missing dates (e.g., filling with mean, median, or artificial timestamps) can introduce bias and create unreal or misleading sequences in the data.

- For this reason, no imputation or replacement is applied to datetime columns.

- Missing datetime values are kept as they are to preserve analytical integrity and avoid distorting time-based trends.


# 6. Correlations
- The goal of this step is to identify the relationships between the variables within the dataset.

- Understanding these correlations helps guide a more targeted and meaningful analysis, revealing how different features interact and potentially influence one another.

- A correlation heatmap will be used to visually examine these relationships, allowing for a clearer interpretation of both the strength and direction of the associations.

In [ ]:
dfs = {
    "customers": customers,
    "geolocation": geolocation,
    "order_items": orders_items,
    "order_payments": order_payments,
    "order_reviews": order_reviews,
    "order": order,
    "products": products,
    "sellers": sellers
}


for name, df in dfs.items():
    numeric_df = df.select_dtypes(include=[float, int])
    print(f"\nCorrelation matrix for {name}:")
    print(numeric_df.corr())



# 6.1 Heatmap




In [ ]:
for name, df in dfs.items():
    numeric_df = df.select_dtypes(include=[float, int])
    corr = numeric_df.corr()

    if corr.empty:
        print(f"\nSkipping {name} because correlation matrix is empty or no numeric data.")
        continue

    plt.figure(figsize=(18,4))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5)
    plt.title(f'Correlation Matrix Heatmap for {name}')
    plt.show()



## 6.2 Heatmap Insights

Overall, the dataset does not exhibit strong linear relationships among most variables, suggesting that the features operate largely independently and no single variable overwhelmingly drives another.

A few moderate correlations are observed, which align with expected business logic:

- Order value and freight value show moderate relationships.
This makes sense: larger orders tend to weigh more or require more volume, which can increase freight costs and total order value.

- Payments installments is moderately correlated with payment_value, indicating that higher payment value number of installments tends to be higher as well.

- Product height and width correlate with product weight (grams), which is consistent with basic physical and volumetric characteristics of merchandise.

These correlations, while not strong, provide useful signals that validate the internal consistency of the dataset and highlight logical patterns in customer orders, product attributes, and orders payments details.


# 7. Summary


- Data Quality: The datasets overall exhibit good quality with manageable missing data and minimal duplicates.

- Missing Data: While some missing values exist, particularly in datetime fields, these were preserved intentionally to avoid introducing bias through imputation or deletion, The majority of datetime nulls are likely because of order_status, further in sql analysis it'll be analyzed.

- Categorical missing values, where appropriate, were replaced with 0 to maintain dataset integrity.

- Outliers: Exploratory visualizations such as boxplots revealed no significant outliers that could potentially distort analyses or modeling outcomes, indicating the dataset's consistency.

- Duplicates: A thorough check showed few duplicate records in one dataset; these will be considered later in sql advanced analysis

- Correlations: Correlation heatmaps highlighted moderate relationships among numeric variables, offering insights for further analysis.

- Data Cleaning: Irrelevant or redundant columns with high missing rates or little analytical value were dropped.

- Next Steps: The cleaned datasets are ready for advanced analysis, Tableau dashboards and reporting.
This thorough cleaning and exploration forms a robust foundation for reliable and effective data-driven decision-making.

# 8 Export data to csv

In [ ]:
geolocation.to_csv('geolocation.csv', index=False)
order_payments.to_csv('order_payments.csv', index=False)
products.to_csv('products.csv', index=False)
order.to_csv('orders.csv', index=False)
orders_items.to_csv('order_items.csv', index=False)
customers.to_csv('customers.csv', index=False)
sellers.to_csv('sellers.csv', index=False)
order_reviews.to_csv('order_reviews.csv', index=False)


In [ ]:
from google.colab import files

files.download('geolocation.csv')
files.download('order_payments.csv')
files.download('products.csv')
files.download('orders.csv')
files.download('order_items.csv')
files.download('customers.csv')
files.download('sellers.csv')
files.download('order_reviews.csv')
